# 3547 TERM PROJECT

## Intelligent Systems and Reinforcement Learning

### Title: Robot obstacle avoidance with reinforcement learning

#### Group Members:
    1. Alexandre Dietrich
    2. Ankur Tyagi
    3. Haitham Alamri
    4. Rodolfo Vasconcelos


Date: August, 4 2020

# Table of Contents

* [Introduction](#Introduction)
* [Problem](#Problem)
* [Solution](#Solution)
    * [Training Phase](#Training_Phase)
    * [Playing Phase](#Playing_Phase)
* [Alternatives](#Alternatives)
* [Evaluation](#Evaluation)
* [Conclusion](#Conclusion)

<a id='Introduction'></a>
# Introduction

After ten weeks learning additional Artificial Intelligence techniques that cover more traditional AI branchs like search, planning, knowledge, logic and reinforcement learning, we had an opportunity to put what we have learned in practices. After interesting brainstorming and debates, we decided to go forward with a project using reinforcement learning. After finding an interesting problem and with the right size to accomplish a solution in a razonable time and resources, we put our efforts to define the problem, define alternatives solutions, code and test our algoritms, evaluate the results and prepare the conclusion. 

<a id='Problem'></a>
# Problem

### Problem statement: find the optimal path for a robot to reach point B starting from point A, avoiding all obstacles during its journey, using reinforcement learning methods

We found an implementation of this problem which uses other techniques instead of RL. We called it the "Robot World".

<div align="center">
<img src="original_simulator.png" alt="Drawing" style="width: 800px;"/>
<center><b>Figure 1. Original Robot World Simulator</b></center>
<center><i>Source: XXXXXX</i></center>
</div> 

In this robot world, you can move by clicking on the step button. After a new step, the robot will be in a new state. You can include new obstacles during the journey dynamically. The objective is to reach the goal point (G) without bumbing in an obstacle. You can also define the starting and goal points. 

The original code implemented three important classes: Robot, Sonar and Sonar_Array. The first one controls the robot, is position, its moves, etc. The other two implement the controls to get the information from the all the sensors to uderstantd if there are obstacles in a close position, define the alerts and define the actions that controlif the robot should contunue in its trajectory or need to make a turn to avoid the obstacle. This part of the code is where our main problem resides. How can we use reinforcement learning to define an optimal policy to guide the actions to avoid the obstacles and reach the goal ?  

The original code and simultador can be see and run in this link: <br>
http://www.codeskulptor.org/#user40_EEIxkOtKog_1.py

To effectviley use the original code and simulator, we made some changes and adapted the Robot World. Our objective was to focus more in the reinforcement learning than in other parts of the code. Here are the main changes:

New Robot World Assuptions: <br>
500 x 500 pyxels <br>
Start point at the top left corner <br>
End point at the bottom right corner <br>

<div align="center">
<img src="new_simulator.png" alt="Drawing" style="width: 800px;"/>
<center><b>Figure 2. New Robot World Simulator</b></center>
<center><i>Source: XXXXXX</i></center>
</div> 

Following is the definition of the Markov decision process: 

States: the set of states  (XXXXXXX)<br>
Start State: top left corner<br>
Actions(s): Up, Down, Right and left<br>
T(s'|s; a): probability of s0 if take action a in state s = 1 (No uncertainty)<br>
Reward(s; a; s0): reward for the transition (s; a; s0) = XXXXXX<br>
End State: bottom right corner <br>
Discount factor = 0.7<br>

# Solution

Our solution departed from the assuptions we have made to adapt to a new Robot World. In our journey to find an optimal policy to help our robot to achieve its goal without bumping in obstacles, we took an operational decision and divided the New Robot World (500 x 500) in 100 grids of 4 x 4 positions. With that, we created a strategy to find the best policy inside a smaller grid. After finding the policy which provide the robot to a optimal path inside this 4 x 4 grid, we can use it to any other grid inside the New Robot World. 

<a id='Training_Phase'></a>
## Training Phase

To build the master policy, we use Monte Carlo algorithm to simulate thousands of episodes and value iteration to find the optimal q-value fucntion for every state and action. 

With the Master Policy trained, we adapted the original code of the Robot World to our create our New Robot World. We included the Master Policy as a dictionary considering States and Actions. We also created to functions: find_location_onMap and policy_finder, which are use to get information of the robot and obstacles inside an specific grid and define the optimal actions according the these positions. 

# Insert below the code that generates the Master Policy

In [ ]:
import numpy as np
import random

master_policy={}

class Grid: # Environment
  def __init__(self, width, height, start):
    self.width = width
    self.height = height
    self.i = start[0]
    self.j = start[1]

  def set(self, rewards, actions):
    # rewards should be a dict of: (i, j): r (row, col): reward
    # actions should be a dict of: (i, j): A (row, col): list of possible actions
    self.rewards = rewards
    self.actions = actions

  def set_state(self, s):
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    return (self.i, self.j)

  def is_terminal(self, s):
    return s not in self.actions

  def move(self, action):
    # check if legal move first
    if action in self.actions[(self.i, self.j)]:
      same_state = (self.i,self.j)
      if action == 'U':
        self.i -= 1
      elif action == 'D':
        self.i += 1
      elif action == 'R':
        self.j += 1
      elif action == 'L':
        self.j -= 1
    # # if agent hit the wall go back to same position.
    if (self.i==4 or self.i==-1):
      (self.i,self.j)=same_state
    if (self.j==4 or self.j==-1):
      (self.i,self.j)=same_state
        # return a reward (if any)
    return self.rewards.get((self.i, self.j), 0)

  def game_over(self):
    # returns true if game is over, else false
    # true if we are in end state (0,0) or (3,3)
    end_state = [(3,3)]
    return (self.i, self.j) in end_state

  def all_states(self):
    # either a position that has possible next actions
    # or a position that yields a reward
    return set(self.actions.keys()) | set(self.rewards.keys())


def standard_grid(starting_position):
  # define a grid that describes the reward for arriving at each state
  # and possible actions at each state
  g = Grid(4, 4, starting_position)
  rewards = {
    (3, 3): 0,
    (k1,k2):-5,
  }
  actions = {
    (0, 0): ('L', 'D', 'R','U'),
    (0, 1): ('L', 'D', 'R','U'),
    (0, 2): ('L', 'D', 'R','U'),
    (0, 3): ('L', 'D', 'R','U'),
    (1, 0): ('L', 'D', 'R','U'),
    (1, 1): ('L', 'D', 'R','U'),
    (1, 2): ('L', 'D', 'R','U'),
    (1, 3): ('L', 'D', 'R','U'),
    (2, 0): ('L', 'D', 'R','U'),
    (2, 1): ('L', 'D', 'R','U'),
    (2, 2): ('L', 'D', 'R','U'),
    (2, 3): ('L', 'D', 'R','U'),
    (3, 0): ('L', 'D', 'R','U'),
    (3, 1): ('L', 'D', 'R','U'),
    (3, 2): ('L', 'D', 'R','U'),
    (3, 3): ('L', 'D', 'R','U'),
  }
  g.set(rewards, actions)
  return g


def negative_grid(step_cost=-1):
  # we minimize the number of moves
  # so we will penalize every move (-1) except for the end-state (0)
  
  states = [[i, j] for i in range(4) for j in range(4)]
  initState = random.choice(states[1:-1])
  g = standard_grid(initState)
  g.rewards.update({
    (0, 0): step_cost,
    (0, 1): step_cost,
    (0, 2): step_cost,
    (0, 3): step_cost,
    (1, 0): step_cost,
    (1, 1): step_cost,
    (1, 2): step_cost,
    (1, 3): step_cost,
    (2, 0): step_cost,
    (2, 1): step_cost,
    (2, 2): step_cost,
    (2, 3): step_cost,
    (3, 0): step_cost,
    (3, 1): step_cost,
    (3, 2): step_cost,
    (3, 3): 0,
    (k1,k2): -5,

  })
  return g


def print_values(V, g):
  for i in range(g.width):
    print("---------------------------")
    for j in range(g.height):
      v = V.get((i,j), 0)
      if v >= 0:
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="") # -ve sign takes up an extra space
    print("")


def print_policy(P, g):
  for i in range(g.width):
    print("---------------------------")
    for j in range(g.height):
      a = P.get((i,j), ' ')
      print("  %s  |" % a, end="")
    print("")
# the Monte Carlo Epsilon-Greedy method to find the optimal policy and value function
import numpy as np
import matplotlib.pyplot as plt

GAMMA = 0.7
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')

def random_action(a, eps=0.1):
  # choose given a with probability 1 - eps + eps/4
  p = np.random.random()
  if p < (1 - eps):
    return a
  else:
    return np.random.choice(ALL_POSSIBLE_ACTIONS)

def max_dict(d):
  # returns the argmax (key) and max (value) from a dictionary
  max_key = None
  max_val = float('-inf')
  for k, v in d.items():
    if v > max_val:
      max_val = v
      max_key = k
  return max_key, max_val

def play_game(grid, policy):
  # returns a list of states and corresponding returns
  # use an epsilon-soft policy
  # Random start position
  states = [(i, j) for i in range(4) for j in range(4)]
  initState = random.choice(states[1:-1])
  g = standard_grid(initState)
  s = initState
  grid.set_state(s)
  a = random_action(policy[s])

  # each triple is s(t), a(t), r(t)
  # but r(t) results from taking action a(t-1) from s(t-1) and landing in s(t)
  states_actions_rewards = [(s, a, 0)]
  while True:
    r = grid.move(a)
    s = grid.current_state()
    if grid.game_over():
      states_actions_rewards.append((s, None, r))
      break
    else:
      a = random_action(policy[s]) # the next state is stochastic
      states_actions_rewards.append((s, a, r))

  # calculate the returns by working backwards from the terminal state
  G = 0
  states_actions_returns = []
  first = True
  for s, a, r in reversed(states_actions_rewards):
    # the value of the terminal state is 0 
    # we should ignore the first state we encounter
    # and ignore the last G, which is meaningless since it doesn't correspond to any move
    if first:
      first = False
    else:
      states_actions_returns.append((s, a, G))
    G = r + GAMMA*G
  states_actions_returns.reverse() # we want it to be in order of state visited
  return states_actions_returns

for i in range(0,4):
  master_policy[i]={}
  for i1 in range(0,4):
    if (i==0 & i1==0):
      k1=2
      k2=2
    elif (i==3 & i1==3):
      k1=2
      k2=2
    else:
      k1=i
      k2=i1
    grid = negative_grid(step_cost=-1)
# print rewards
    print("rewards:")
    print_values(grid.rewards, grid)

# state -> action
# initialize a random policy
    policy = {}
    for s in grid.actions.keys():
      policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
  
# initial policy
    print("initial policy:")
    print_policy(policy, grid)

# initialize Q(s,a) and returns
    Q = {}
    returns = {} # dictionary of state -> list of returns we've received
    states = grid.all_states()
    for s in states:
      if s in grid.actions: # not a terminal state
        Q[s] = {}
        for a in ALL_POSSIBLE_ACTIONS:
          Q[s][a] = 0
          returns[(s,a)] = []
      else:
    # terminal state or state we can't otherwise get to
        pass
  
# initial Q values for all states in grid
    print(Q)
    print(s)



# repeat
    deltas = []
    for t in range(1000):
  # generate an episode using pi
      biggest_change = 0
      states_actions_returns = play_game(grid, policy)

  # calculate Q(s,a)
      seen_state_action_pairs = set()
      for s, a, G in states_actions_returns:
    # check if we have already seen s
    # called "first-visit" MC policy evaluation
        sa = (s, a)
        if sa not in seen_state_action_pairs:
          old_q = Q[s][a]
          returns[sa].append(G)
          Q[s][a] = np.mean(returns[sa])
          biggest_change = max(biggest_change, np.abs(old_q - Q[s][a]))
          seen_state_action_pairs.add(sa)
      deltas.append(biggest_change)

  # calculate new policy pi(s) = argmax[a]{ Q(s,a) }
      for s in policy.keys():
        a, _ = max_dict(Q[s])
        policy[s] = a


    plt.plot(deltas)
    plt.show()

# find the optimal state-value function
# V(s) = max[a]{ Q(s,a) }
    V = {}
    for s in policy.keys():
      V[s] = max_dict(Q[s])[1]

# Print the table of the estimated function Q(s,a) for the optimal policy 
    print("final values:")
    print_values(V, grid)

    print("final policy:")
# Print the policy pi(s) 
    print_policy(policy, grid)
    print(policy)
    master_policy[i][i1]={}
    master_policy[i][i1]=policy
    print("#########################")
    print(i,i1)
    print("#######################")

print (master_policy)

def find_location_onMap(pos):
  location_in_the_grid=[]
  location_in_the_map=[]
  i=0
  i1=0
  loc=0
  loc1=0
  for squares in range(0,10):
    i1=0
    for quare in range(0,10):
      loc=0
      #square.append((i,i1))
      if ((pos[0]>i and pos[0]<=i+50) and (pos[1]>i1 and pos[1]<=i1+50)):
        for rows in range(0,4):
          loc1=0
          for col in range(0,4):
            if ((pos[0]>loc+i and pos[0]<=loc+i+12.5) and (pos[1]>loc1+i1 and pos[1]<=loc1+12.5+i1)):           
              location_in_the_grid.append(loc)
              location_in_the_grid.append(loc1)
              location_in_the_map.append(i)
              location_in_the_map.append(i1)
            
            loc1+=12.5
          loc+=12.5
      i1+=50
    i+=50

#To to convert to 3x3 grid
  location_in_the_grid[0]=int(location_in_the_grid[0]/12.5)
  location_in_the_grid[1]=int(location_in_the_grid[1]/12.5)
#To to convert to 5x5 map each sqaure is 100X100 pixel
  location_in_the_map[0]=int(location_in_the_map[0]/50)
  location_in_the_map[1]=int(location_in_the_map[1]/50)
  return location_in_the_map, location_in_the_grid

def policy_finder (mylocation,obs):
    policy= master_policy[obs_location_onGrid[0]][obs_location_onGrid[1]]
    direction = policy.get((my_location_onGrid[0],my_location_onGrid[1]), ' ')
    print(direction)
    return direction
  


# This function is to check if the obtacles and agent are in the same 12.5x12.5 pixels sqaure
def check_obstacle(obs, obs_list):
  location_on_map,location_on_grid = find_location_onMap(obs)
  print(location_on_map)
  for i in full_obstacle_list:
    location_on_map1,location_on_grid1 = find_location_onMap(i)
    if (location_on_map1 == location_on_map):
        return location_on_map
  return None

# Master policy for 2 Obstacles

import numpy as np
import random
from collections import defaultdict

master_policy={}

class Grid: # Environment
  def __init__(self, width, height, start):
    self.width = width
    self.height = height
    self.i = start[0]
    self.j = start[1]

  def set(self, rewards, actions):
    # rewards should be a dict of: (i, j): r (row, col): reward
    # actions should be a dict of: (i, j): A (row, col): list of possible actions
    self.rewards = rewards
    self.actions = actions

  def set_state(self, s):
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    return (self.i, self.j)

  def is_terminal(self, s):
    return s not in self.actions

  def move(self, action):
    # check if legal move first
    if action in self.actions[(self.i, self.j)]:
      same_state = (self.i,self.j)
      if action == 'U':
        self.i -= 1
      elif action == 'D':
        self.i += 1
      elif action == 'R':
        self.j += 1
      elif action == 'L':
        self.j -= 1
    # # if agent hit the wall go back to same position.
    if (self.i==4 or self.i==-1):
      (self.i,self.j)=same_state
    if (self.j==4 or self.j==-1):
      (self.i,self.j)=same_state
        # return a reward (if any)
    return self.rewards.get((self.i, self.j), 0)

  def game_over(self):
    # returns true if game is over, else false
    # true if we are in end state (0,0) or (3,3)
    end_state = [(3,3)]
    return (self.i, self.j) in end_state

  def all_states(self):
    # either a position that has possible next actions
    # or a position that yields a reward
    return set(self.actions.keys()) | set(self.rewards.keys())


def standard_grid(starting_position,k1,k2,x1,x2):
  # define a grid that describes the reward for arriving at each state
  # and possible actions at each state
  g = Grid(4, 4, starting_position)
  rewards = {
    (3, 3): 0,
    (k1,k2):-5,
    (x1,x2):-5,
  }
  actions = {
    (0, 0): ('L', 'D', 'R','U'),
    (0, 1): ('L', 'D', 'R','U'),
    (0, 2): ('L', 'D', 'R','U'),
    (0, 3): ('L', 'D', 'R','U'),
    (1, 0): ('L', 'D', 'R','U'),
    (1, 1): ('L', 'D', 'R','U'),
    (1, 2): ('L', 'D', 'R','U'),
    (1, 3): ('L', 'D', 'R','U'),
    (2, 0): ('L', 'D', 'R','U'),
    (2, 1): ('L', 'D', 'R','U'),
    (2, 2): ('L', 'D', 'R','U'),
    (2, 3): ('L', 'D', 'R','U'),
    (3, 0): ('L', 'D', 'R','U'),
    (3, 1): ('L', 'D', 'R','U'),
    (3, 2): ('L', 'D', 'R','U'),
    (3, 3): ('L', 'D', 'R','U'),
  }
  g.set(rewards, actions)
  return g


def negative_grid(step_cost,k1,k2,x1,x2):
  # we minimize the number of moves
  # so we will penalize every move (-1) except for the end-state (0)
  
  states = [[i, j] for i in range(4) for j in range(4)]
  initState = random.choice(states[1:-1])
  g = standard_grid(initState,k1,k2,x1,x2)
  g.rewards.update({
    (0, 0): step_cost,
    (0, 1): step_cost,
    (0, 2): step_cost,
    (0, 3): step_cost,
    (1, 0): step_cost,
    (1, 0): step_cost,
    (1, 1): step_cost,
    (1, 2): step_cost,
    (1, 3): step_cost,
    (2, 0): step_cost,
    (2, 1): step_cost,
    (2, 2): step_cost,
    (2, 3): step_cost,
    (3, 0): step_cost,
    (3, 1): step_cost,
    (3, 2): step_cost,
    (3, 3): 0,
    (k1,k2): -5,
    (x1,x2): -5,

  })
  return g


def print_values(V, g):
  for i in range(g.width):
    print("---------------------------")
    for j in range(g.height):
      v = V.get((i,j), 0)
      if v >= 0:
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="") # -ve sign takes up an extra space
    print("")


def print_policy(P, g):
  for i in range(g.width):
    print("---------------------------")
    for j in range(g.height):
      a = P.get((i,j), ' ')
      print("  %s  |" % a, end="")
    print("")
# the Monte Carlo Epsilon-Greedy method to find the optimal policy and value function
import numpy as np
import matplotlib.pyplot as plt

GAMMA = 0.7
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')

def random_action(a, eps=0.1):
  # choose given a with probability 1 - eps + eps/4
  p = np.random.random()
  if p < (1 - eps):
    return a
  else:
    return np.random.choice(ALL_POSSIBLE_ACTIONS)

def max_dict(d):
  # returns the argmax (key) and max (value) from a dictionary
  max_key = None
  max_val = float('-inf')
  for k, v in d.items():
    if v > max_val:
      max_val = v
      max_key = k
  return max_key, max_val

def play_game(grid, policy):
  # returns a list of states and corresponding returns
  # use an epsilon-soft policy
  # Random start position
  states = [(i, j) for i in range(4) for j in range(4)]
  initState = random.choice(states[1:-1])
  g = standard_grid(initState,k1,k2,x1,x2)
  s = initState
  grid.set_state(s)
  a = random_action(policy[s])

  # each triple is s(t), a(t), r(t)
  # but r(t) results from taking action a(t-1) from s(t-1) and landing in s(t)
  states_actions_rewards = [(s, a, 0)]
  while True:
    r = grid.move(a)
    s = grid.current_state()
    if grid.game_over():
      states_actions_rewards.append((s, None, r))
      break
    else:
      a = random_action(policy[s]) # the next state is stochastic
      states_actions_rewards.append((s, a, r))

  # calculate the returns by working backwards from the terminal state
  G = 0
  states_actions_returns = []
  first = True
  for s, a, r in reversed(states_actions_rewards):
    # the value of the terminal state is 0 
    # we should ignore the first state we encounter
    # and ignore the last G, which is meaningless since it doesn't correspond to any move
    if first:
      first = False
    else:
      states_actions_returns.append((s, a, G))
    G = r + GAMMA*G
  states_actions_returns.reverse() # we want it to be in order of state visited
  return states_actions_returns

for i in range(0,4):
  master_policy[i]={}
  for i1 in range(0,4):
    if (i==3 & i1==3):
      k1=2
      k2=2
    else:
      k1=i
      k2=i1
    master_policy[i][i1]={}
    for s0 in range(0,4):
      master_policy[i][i1][s0]={}
      for s1 in range(0,4): 
        if (s0==3 & s1==3):
          x1=2
          x2=2
        else:
          x1=s0
          x2=s1
        grid = negative_grid(-1,k1,k2,x1,x2)
# print rewards
        print("rewards:")
        print_values(grid.rewards, grid)

# state -> action
# initialize a random policy
        policy = {}
        for s in grid.actions.keys():
          policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
  
# initial policy
        print("initial policy:")
        print_policy(policy, grid)

# initialize Q(s,a) and returns
        Q = {}
        returns = {} # dictionary of state -> list of returns we've received
        states = grid.all_states()
        for s in states:
          if s in grid.actions: # not a terminal state
            Q[s] = {}
            for a in ALL_POSSIBLE_ACTIONS:
              Q[s][a] = 0
              returns[(s,a)] = []
          else:
    # terminal state or state we can't otherwise get to
            pass
  
# initial Q values for all states in grid
        print(Q)
        print(s)



# repeat
        deltas = []
        for t in range(1000):
  # generate an episode using pi
          biggest_change = 0
          states_actions_returns = play_game(grid, policy)

  # calculate Q(s,a)
          seen_state_action_pairs = set()
          for s, a, G in states_actions_returns:
    # check if we have already seen s
    # called "first-visit" MC policy evaluation
            sa = (s, a)
            if sa not in seen_state_action_pairs:
              old_q = Q[s][a]
              returns[sa].append(G)
              Q[s][a] = np.mean(returns[sa])
              biggest_change = max(biggest_change, np.abs(old_q - Q[s][a]))
              seen_state_action_pairs.add(sa)
          deltas.append(biggest_change)

  # calculate new policy pi(s) = argmax[a]{ Q(s,a) }
          for s in policy.keys():
            a, _ = max_dict(Q[s])
            policy[s] = a


        plt.plot(deltas)
        plt.show()

# find the optimal state-value function
# V(s) = max[a]{ Q(s,a) }
        V = {}
        for s in policy.keys():
          V[s] = max_dict(Q[s])[1]

# Print the table of the estimated function Q(s,a) for the optimal policy 
        print("final values:")
        print_values(V, grid)

        print("final policy:")
# Print the policy pi(s) 
        print_policy(policy, grid)
        print(policy)
        print("#########################")
        print(i,i1,s0,s1)
        master_policy[i][i1][s0][s1]={}
        master_policy[i][i1][s0][s1]=policy
        print("#########################")
       
      

print (master_policy)

full_obstacle_list = [(10,200),(250, 110), (200,280),(220,200), (300,300), (200,100), (400,150),(400, 110), 
                 (430, 300), (201, 304), (135, 281), 
                 (286, 373), (175, 280), (250, 375), (139, 327), (369, 278), 
                 (295, 196),(40,45),(60, 100),(210, 111),(100,100),(110,110),(79,250)]
pos=(430, 390)
print(check_obstacle(pos,full_obstacle_list))

def check_obstacle(obs, obs_list):
  location_on_map,location_on_grid = find_location_onMap(obs)
  print(location_on_map)
  for i in full_obstacle_list:
    location_on_map1,location_on_grid1 = find_location_onMap(i)
    if (location_on_map1 == location_on_map):
      print(i)
      print(location_on_map1)
      return True
  return False
    


<a id='Playing_Phase'></a>
## Playing Phase

With the Master Policy trained, we adapted the original code of the Robot World to our create our New Robot World. We included the Master Policy as a dictionary considering States and Actions. We also created to functions: find_location_onMap and policy_finder, which are use to get information of the robot and obstacles inside an specific grid and define the optimal actions according the these positions. 

Inside the code where the original program understand what kind of actions the robot should take, we change the code to call our policy_finder fuction, which discovers the robot and obstacle locations, and return the action that the robot must take, according our Master Policy. 

Now you can play with the New Robot World clicking on the step button and adding new obstacles. 

Here is the link to the New Robot World:

http://www.codeskulptor.org/#user47_JJSgNIMtcq_0.py

Following is the modified code that change the original simulator to the New Robot World.

# Insert below the code that plays with the robot to avoid obstacles with RL

In [ ]:
import simplegui
import math
import random

#define constants


# New Code

master_policy={0: {0: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'D', (1, 1): 'L', (1, 2): 'U', (1, 3): 'D', (2, 0): 'R', (2, 1): 'D', (2, 2): 'D', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 1: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'D', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'D', (2, 1): 'L', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 2: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'D', (0, 3): 'D', (1, 0): 'U', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'R', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D', (3, 0): 'U', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 3: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'U', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'U', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}}, 1: {0: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'R', (1, 1): 'D', (1, 2): 'R', (1, 3): 'D', (2, 0): 'D', (2, 1): 'D', (2, 2): 'D', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 1: {(0, 0): 'D', (0, 1): 'L', (0, 2): 'R', (0, 3): 'D', (1, 0): 'D', (1, 1): 'D', (1, 2): 'D', (1, 3): 'D', (2, 0): 'D', (2, 1): 'D', (2, 2): 'D', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 2: {(0, 0): 'D', (0, 1): 'L', (0, 2): 'L', (0, 3): 'D', (1, 0): 'D', (1, 1): 'D', (1, 2): 'R', (1, 3): 'D', (2, 0): 'R', (2, 1): 'D', (2, 2): 'L', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 3: {(0, 0): 'R', (0, 1): 'D', (0, 2): 'D', (0, 3): 'L', (1, 0): 'D', (1, 1): 'D', (1, 2): 'D', (1, 3): 'L', (2, 0): 'D', (2, 1): 'R', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}}, 2: {0: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'D', (0, 3): 'D', (1, 0): 'R', (1, 1): 'R', (1, 2): 'D', (1, 3): 'D', (2, 0): 'D', (2, 1): 'R', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 1: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'U', (1, 1): 'R', (1, 2): 'D', (1, 3): 'D', (2, 0): 'D', (2, 1): 'L', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 2: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'D', (1, 1): 'U', (1, 2): 'R', (1, 3): 'D', (2, 0): 'D', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 3: {(0, 0): 'R', (0, 1): 'D', (0, 2): 'D', (0, 3): 'L', (1, 0): 'U', (1, 1): 'D', (1, 2): 'D', (1, 3): 'L', (2, 0): 'U', (2, 1): 'R', (2, 2): 'D', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}}, 3: {0: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'D', (0, 3): 'D', (1, 0): 'U', (1, 1): 'U', (1, 2): 'D', (1, 3): 'D', (2, 0): 'U', (2, 1): 'D', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 1: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'R', (0, 3): 'D', (1, 0): 'U', (1, 1): 'D', (1, 2): 'R', (1, 3): 'D', (2, 0): 'R', (2, 1): 'R', (2, 2): 'D', (2, 3): 'D', (3, 0): 'U', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}, 2: {(0, 0): 'R', (0, 1): 'R', (0, 2): 'D', (0, 3): 'D', (1, 0): 'R', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'U', (2, 1): 'U', (2, 2): 'R', (2, 3): 'D', (3, 0): 'R', (3, 1): 'U', (3, 2): 'R', (3, 3): 'U'}, 3: {(0, 0): 'D', (0, 1): 'D', (0, 2): 'R', (0, 3): 'D', (1, 0): 'R', (1, 1): 'R', (1, 2): 'R', (1, 3): 'D', (2, 0): 'U', (2, 1): 'D', (2, 2): 'D', (2, 3): 'D', (3, 0): 'U', (3, 1): 'R', (3, 2): 'R', (3, 3): 'U'}}}

# New Code


OBSTACLE_RAD = 20 # how big (radius) are the obstacles
ROBOT_RAD = 30 # how big (radius) is the robot?
SENSOR_FOV = 10.0 # FOV of each sensor THIS MUST BE A FLOAT!!!!!! 
SENSOR_MAX_R = 100 # max range that each sensor can report
SENSOR_ALERT_R = 50 #range within which sensor reports are acted upon
TURN_SCALE_FACTOR = 2 # how drastic do we want the turns to be
SAFETY_DISTANCE = 40
#helper functions


# New Code


def find_location_onMap(pos):
  location_in_the_grid=[]
  location_in_the_map=[]
  i=0
  i1=0
  loc=0
  loc1=0
  for squares in range(0,10):
    i1=0
    for quare in range(0,10):
      loc=0
      #square.append((i,i1))
      if ((pos[0]>i and pos[0]<=i+50) and (pos[1]>i1 and pos[1]<=i1+50)):
        for rows in range(0,4):
          loc1=0
          for col in range(0,4):
            if ((pos[0]>loc+i and pos[0]<=loc+i+12.5) and (pos[1]>loc1+i1 and pos[1]<=loc1+12.5+i1)):           
              location_in_the_grid.append(loc)
              location_in_the_grid.append(loc1)
              location_in_the_map.append(i)
              location_in_the_map.append(i1)
            
            loc1+=12.5
          loc+=12.5
      i1+=50
    i+=50

#To to convert to 3x3 grid
  location_in_the_grid[0]=int(location_in_the_grid[0]/12.5)
  location_in_the_grid[1]=int(location_in_the_grid[1]/12.5)
#To to convert to 5x5 map each sqaure is 100X100 pixel
  location_in_the_map[0]=int(location_in_the_map[0]/50)
  location_in_the_map[1]=int(location_in_the_map[1]/50)
  return location_in_the_map, location_in_the_grid


def policy_finder (mylocation,obs):
  mylocation_onMap, my_location_onGrid = find_location_onMap(mylocation)
  obs_location_onMap, obs_location_onGrid = find_location_onMap(obs)
#  print("My location: ", mylocation_onMap)
#  print("Obstacle location: ", obs_location_onMap)
  if (mylocation_onMap == obs_location_onMap):
    print("Match found")
    policy= master_policy[obs_location_onGrid[0]][obs_location_onGrid[1]]
    direction = policy.get((my_location_onGrid[0],my_location_onGrid[1]), ' ')
    print(direction)
    return direction
  else:
    return None

# New Code


def rel_brg_fm_offset_sensor(true_hdg, sensor_offset, tgt_brg):
    #given robot's true heading, the sensor offset angle and the
    #true brg of the target, this fn will return the relative brg
    #of the target from the sensor's line of sight
    
    sensor_look_brg = (true_hdg + sensor_offset)%360
    tgt_rel_fm_sensor = tgt_brg - sensor_look_brg

    if tgt_rel_fm_sensor < -180:
        tgt_rel_fm_sensor += 360
    
    return tgt_rel_fm_sensor

def brg_in_deg(p0, p1):#bearing only in degrees
    [x1, y1] = p0
    [x2, y2] = p1
    a = math.degrees(math.atan((y1 - y2)/(x1 - x2 + 0.000000001)))
    #find and correct the quadrant...
    if  x2 >= x1:
        b = 90 + a
    else:
        b = 270 + a
    return b

def dist(p1, p0):#distance only
    return math.sqrt((p1[0] - p0[0])**2+(p1[1]-p0[1])**2)

def dist_and_brg_in_deg(p0, p1):#bearing and distance in degrees between two points
    [x1, y1] = p0
    [x2, y2] = p1
    r = math.sqrt((x1 - x2)**2 + (y1 - y2)**2) # distance
    a = math.degrees(math.atan((y1 - y2)/(x1 - x2 + 0.000000001)))
    #find and correct the quadrant...
    if  x2 >= x1:
        b = 90 + a
    else:
        b = 270 + a
    return r, b

def angle_to_vector(ang):#resolve angles into vectors
    ang = math.radians(ang)
    return [math.cos(ang), math.sin(ang)]

def relative_brg(b1, b2):
    rb = b2 - b1
    if rb > 180:
        rb = 360 - rb
    if rb < -180:
        rb += 360
        rb *= -1
    return rb

def create_vector(from_pos, length, brg):       
        u_vec = angle_to_vector(brg)
        #print "generating target line..."
        vec0 = from_pos[0] + length * u_vec[1] 
        vec1 = from_pos[1] - length * u_vec[0]
        
        return [vec0, vec1]

    
#define classes

class Sonar:
    def __init__(self, index, FOV, max_r, robot_co):
        #create a instance of this class
        self.pos = [0,0]
        self.index = index
        self.max_r = max_r
        self.offset =  index * FOV #+ FOV/2 # on what relative bearing is this sensor looking?
        self.look_brg = (robot_co + self.offset)%360
        self.vec = [0,0] # just a vector for grpahical ouptut of pings
        self.has_valid_echo = False #indicates if this sonar has a "valid" obstacle in sight
        #print "Creating Sonar:" , index, " offset ", self.offset, "true LOS:", self.look_brg
            
    def ping_actual():#ping for real in a robot and return range observed
        pass
    
    def ping_simulated(self, obstacle_list, robot_co):
        #from robot position and robot_co, run through obs_list
        #return the distance to closest object within 
        #FOV and within self.max_r of THIS SENSOR
        # range all the obstacles in view, find the nearest
        
        range_list = []
        
        for obs in obstacle_list:# find objects within max_r and inside FOV
            #print "pinging for robot_co:", robot_co
            can_observe, d = self.can_observe(robot_co, obs, OBSTACLE_RAD)
            #print can_observe, d
            if can_observe:
                range_list.append(d) 
        if len(range_list) > 0:
            self.output = min(range_list)- SAFETY_DISTANCE
        else:
            self.output = SENSOR_MAX_R
        #print "closest point: " , obs , " distance:", self.output
    
    def get_output(self):
        return self.output
    
    def can_observe(self, robot_co, obstacle_pos, obstacle_rad):
        #if obstacle is within within the max_r of the sensor
        #and within FOV, return True and the distance observed
        #else return False, 0
        
        dist, brg = dist_and_brg_in_deg(self.pos, obstacle_pos)
        
        if dist < self.max_r: #if the object is within max_r....
            rel_brg = rel_brg_fm_offset_sensor(robot_co, self.offset, brg)#rel brg of tgt from sensor LOS
            d_test = abs(dist * math.asin(math.radians(rel_brg)))
            if d_test < OBSTACLE_RAD + ROBOT_RAD: 
                self.has_valid_echo = True
                return True, dist # if the object is within min allowed lateral separation
            else:
                self.has_valid_echo = False
                return False, 0 # ignore it
        else:#if the object is outside max_r of this sonar...ignore it
            self.has_valid_echo = False
            return False, 0
      
    def update(self, platform_pos, platform_co, obstacle_list):
              
        #update own parameters
        self.pos = platform_pos
        
        self.look_brg = (platform_co + self.offset)%360
        
        #calculate output of this sensor
        
        self.ping_simulated(obstacle_list, platform_co)
        
        self.vec = create_vector(self.pos, self.output + ROBOT_RAD, self.look_brg)#calculate distance vector for drawing on canvas
        
        #print "sensor index:", self.index, " look brg:", self.look_brg
    def draw(self, canvas): # draw the sensor's output
        #if self.has_valid_echo:
        canvas.draw_line(self.pos,self.vec, 1, 'lime')
        canvas.draw_text(str(self.index),(self.vec[0]+4, self.vec[1]+4), 10, "lime"),
        #print self.index, " VE:" , self.has_valid_echo
        
class Sonar_Array:
    def __init__(self, n_sensors, SENSOR_FOV, SENSOR_MAX_R, robot_co):
        self.sonar_list = []
        self.need_diversion_flag = False
        
        i_pos = range(1, n_sensor/2 + 1)
        i_neg = range(-n_sensor/2 , 0)
        i_pos.reverse()
        i_neg.reverse()
        i_pos.extend(i_neg)
        
        for i in i_pos:#create a list of individual sonars...
            self.sonar_list.append(Sonar(i, SENSOR_FOV, SENSOR_MAX_R, robot_co))
   
    def update(self, robot_pos, robot_co, obstacle_list, method):
        #update sonar array
        for sonar in self.sonar_list:#update output of each sensor
            sonar.update(robot_pos, robot_co, obstacle_list)
            
        if method == "w_sum":#process data by method of weighted sums
            return self.weighted_sum_method(robot_pos, robot_co)
    
    def weighted_sum_method(self, robot_pos, robot_co):
        #process data by the weighted sum method and 
        #return (1) whether turn is required or not (2) index of recommended sonar LOS to turn to
        sum_d = 0
        sum_wt = 0
        alert = False
        #print "checking all sonars:"      
        for sonar in self.sonar_list:
            #print "sonar:", sonar.index, " range:", sonar.output
            if sonar.output < SENSOR_ALERT_R:#has this sonar found anything in danger zone?
                alert = True
                #print "obstacle found by index ", sonar.index
                for s1 in self.sonar_list: #process the whole array
                    d = int(s1.output)
                    gain = 1#SENSOR_MAX_R/(SENSOR_MAX_R - d)
                    sum_d +=  d
                    sum_wt += s1.index * d * gain
                    #print "I:", s1.index,",D:",int(s1.output), ",sum_D:", sum_d, "sum_wt:",sum_wt
                rec_index = math.ceil(TURN_SCALE_FACTOR * float(sum_wt)/sum_d) #index of sonar with best LOS
                #rec_index = int(TURN_SCALE_FACTOR * float(sum_d)/sum_wt)
                print "Rec index:", rec_index
                if abs(rec_index) > n_sensor/2:
                    print "rec index too large"
                    rec_index = n_sensor/2
#                    if rec_index < 0:
#                        rec_index = -n_sensor/2
#                    else:
#                        rec_index = n_sensor/2
                print "Rec index:", rec_index 
                break # processing completed
            else: #no obstacle in danger zone
                rec_index = 0
                #return robot_co, False
                #print : index = ", sonar.index
        #print "break from loop."
        if rec_index == 0 and alert == True:
            print "alert with no alteration"
            
# New Code
            
            for obs in full_obstacle_list:
                    mylocation=robot_pos
                    mylocation_onMap, my_location_onGrid = find_location_onMap(mylocation)
                    obs_location_onMap, obs_location_onGrid = find_location_onMap(obs)
                    action=None
                    if (mylocation_onMap == obs_location_onMap):
                        action = policy_finder(mylocation,obs)
                        if action != None:
                            if action == 'R':
                                offset=90
                            elif action == 'D':
                                offset=180
                            elif action == 'L':
                                offset=270
                            elif action == 'U':
                                offset = 359
                        break
            if (action !=None):
                print(robot_pos)
                print("*********")
                print((offset%robot_co)+robot_co)
                print("******")
                return (offset%robot_co)+robot_co, True 
            else:
               return robot_co, True    
        
# New Code
        
                
        elif abs(rec_index) > 0: # some alteration recommended
            print "turn recommended"
            offset =  rec_index * SENSOR_FOV #how much is the angular offset  
            
# New Code 
            
            for obs in full_obstacle_list:
                mylocation=robot_pos
                mylocation_onMap, my_location_onGrid = find_location_onMap(mylocation)
                obs_location_onMap, obs_location_onGrid = find_location_onMap(obs)
                action=None
                if (mylocation_onMap == obs_location_onMap):
                    action = policy_finder(mylocation,obs)
                    if action != None:
                        if action == 'R':
                            offset=90
                        elif action == 'D':
                            offset=180
                        elif action == 'L':
                            offset=270
                        elif action == 'U':
                            offset = 359
                    break
            print(robot_pos)
            print("*********")
            print(action)
            print((offset%robot_co)+robot_co)
            print("******")
            if (action==None):
               return robot_co, False 
            return (offset%robot_co), True
        
# New Code        
        
        else:# no diversion needed
            print "no alert no diversion"
            return robot_co, False
    
    def draw(self, canvas):
        for sonar in self.sonar_list:
            sonar.draw(canvas)

class Robot:
    def __init__(self, pos, co, n_sensor):
        self.pos = pos
        self.history = [pos]
        self.co = co
        self.spd = 10 # robot speed in pixels/ step
        self.s_array = Sonar_Array(n_sensor, SENSOR_FOV, SENSOR_MAX_R, self.co)
        self.goal_brg = brg_in_deg(self.pos, goal_pos)
        self.obstacles_in_view = []
    
    def get_obstacles_in_view(self):
        return self.obstacles_in_view
    
    def update(self):
        self.obstacles_in_view = [] #delete all the old obstacles in view
        for obs in full_obstacle_list:
            if dist(self.pos, obs) < SENSOR_MAX_R:
                self.obstacles_in_view.append(obs)
                
        #re-calculate direction to goal
        self.goal_brg = brg_in_deg(self.pos, goal_pos)
        #re-estimate sensor output by weighted sum method
        co1, need_turn = self.s_array.update(self.pos, self.goal_brg, self.obstacles_in_view, "w_sum")
        #print "Path Clear:", self.path_is_clear()
        if self.path_is_clear():#can we reach the goal directly from here?
            self.co = brg_in_deg(self.pos, goal_pos)
            print "path clear. ignoring recommendation"
        elif need_turn: #do we need to turn
            self.co = co1
            print "path not clear. following recommendation"
        else: # path is not fully clear, but there are no immediate obstacles
            pass
            #self.co = brg_in_deg(self.pos, goal_pos)

        #m5e the robot by one step...
        self.move(2)

    def path_is_clear(self):#return True if there is a clear path to the goal
        goal_brg = brg_in_deg(self.pos, goal_pos)
        for obs in self.obstacles_in_view:
            if dist(self.pos, goal_pos) > dist(self.pos, obs):
                d_obs, obs_brg = dist_and_brg_in_deg(self.pos, obs)
                rel_brg = abs(relative_brg(goal_brg, obs_brg))
                d_lateral = abs(d_obs * math.asin(math.radians(rel_brg)))
                if d_lateral < OBSTACLE_RAD + ROBOT_RAD: 
                    return False
        return True
    
    def move(self, dT):
        print("VVVVVBVV")        
        print(self.co)
        u_vec = angle_to_vector(self.co)
        
        self.pos[0] += self.spd * dT * u_vec[1]
        self.pos[1] -= self.spd * dT * u_vec[0]
        
        self.history.append([self.pos[0], self.pos[1]])
        
    def get_pos(self):
        return self.pos
    
    def set_pos(self, pos):
        self.pos = pos
    
    def set_co(self, co):
        self.co = co
        #print "setting robot co:", self.co
    
    def delete_history(self):
        self.history = []
   
    def draw(self, canvas):
        #Draw the robot
        canvas.draw_circle(self.pos, 4, 3, "yellow")
        canvas.draw_text("R", [self.pos[0] + 10, self.pos[1] +10], 16, "yellow")
        #Draw brg line to goal
        self.goal_vec = create_vector(self.pos, 150, self.goal_brg)
        canvas.draw_line(self.pos, self.goal_vec, 2, "teal")
        #Draw current heading vector
        self.co_vec = create_vector(self.pos, 150, self.co)
        canvas.draw_line(self.pos, self.co_vec, 2, "white")
        #draw the output of the sonar array
        self.s_array.draw(canvas)
        #draw the obstacles in view
        for obs in self.obstacles_in_view:
            canvas.draw_circle(obs,2,1, "red")
            canvas.draw_circle(obs,OBSTACLE_RAD, 1, "green") 
        #draw history
        for point in self.history:
            canvas.draw_circle(point,2,2, "lime")
        
        
#define globals

g_state = "None"

start_pos = [10,10]
robot_pos = [10, 10]
robot_co = 1

goal_pos = [500,500]
#obstacle_list = [(300, 213), (310, 124), (250, 110), (300, 230)]


# New Code


full_obstacle_list = [(10,200),(250, 110), (200,280),(220,200), (300,300), (200,100), (400,150),(400, 110), 
                 (430, 390), (201, 304), (135, 281), 
                 (286, 373), (175, 280), (250, 375), (139, 327), (369, 278), 
                 (295, 196), (210, 111),(100,100),(110,110),(79,250)]
                 

# New Code
                 
#create a robot with 6 sensors

n_sensor = 16 

#create a sonar array
#s1 = Sonar_Array(n_sensor, SENSOR_FOV, SENSOR_MAX_R, robot_co)
r1 = Robot(robot_pos, robot_co, n_sensor)

r1.update()
#define event handlers

def click(pos):
    global g_state, start_pos, goal_pos, robot_pos
    if g_state == "Start":
        start_pos = pos
        r1.set_pos(list(pos))
    elif g_state == "Goal":
        goal_pos = pos
        r1.set_co(brg_in_deg(r1.get_pos(), pos))
    elif g_state == "Set Robot":
        r1.set_co(brg_in_deg(r1.get_pos(), pos))
        r1.set_pos(list(pos))
        r1.delete_history()
    elif g_state == "Add Obs":
        full_obstacle_list.append(pos)
        print full_obstacle_list
        #update the robot
    r1.update()
    g_state = "None"
        
def set_start():
    global g_state
    g_state = "Start"
    
def set_goal():
    global g_state
    g_state = "Goal"

def set_robot_pos():
    global g_state
    g_state = "Set Robot"

def alter_co(text):
    r1.set_co(float(text))
    r1.update()
            
def draw(canvas):
    #draw start 
    canvas.draw_circle(start_pos, 4, 3, "red")
    canvas.draw_text("S", [start_pos[0] + 10, start_pos[1] +10], 16, "red")
    #draw goal
    canvas.draw_circle(goal_pos, 4, 3, "green")
    canvas.draw_text("G", [goal_pos[0] + 10, goal_pos[1] +10], 16, "green")
    #draw the obstacles
    for obs in full_obstacle_list:
        canvas.draw_circle(obs,2,1, "red")
        canvas.draw_circle(obs,OBSTACLE_RAD, 1, "white") 
    
    #draw sonar lines...
    r1.draw(canvas)

def step():
        r1.update()

def add_obs():
    global g_state
    g_state = "Add Obs"
    
    
#create simplegui controls

f1 = simplegui.create_frame("Obs Avoidance", 500, 500)
#btn_start = f1.add_button("Set Start", set_start, 100)
btn_goal = f1.add_button("Set Goal", set_goal, 100)
btn_robot = f1.add_button("Set Robot", set_robot_pos, 100)
txt_r_co = f1.add_input("Robot Co", alter_co, 100)
btn_step = f1.add_button("Step", step, 100)

btn_add_obs = f1.add_button("Add Obs", add_obs, 100)

f1.set_draw_handler(draw)
f1.set_mouseclick_handler(click)

#start simplegui

f1.start()

# Alternatives

Describe the alternatives






# Insert below the code for alternatives: different RL methods, new assuptions, etc. 

# Evaluation

Describe the aspects of the performance of the algoritms

# Conclusion

Describe the conclusions

<div align="center">
<img src="robot_movements.png" alt="Drawing" style="width: 800px;"/>
<center><b>Figure 3. Final Robot Movements with RL</b></center>
<center><i>Source: XXXXXX</i></center>
</div> 